## Zadania do wykonania

### Zadanie 1. Zaprojektuj klasę dla $kd$-drzewa.

### Zadanie 2. Zaimplementuj wzorzec projektowy łańcuch odpowiedzialności na przykładzie obsługi żądania _http_ (symulacja), w którym przed faktycznym kodem obsługi błędu ma zostać sprawdzone czy użytkownik może wysłać danego typu żądanie, czy żądanie nie dotyczy pliku, czy liczba żądań na minutę nie jest przekroczona, czy liczba żądań na minutę nie jest przekroczona dla zalogowanego użytkownika, czy przesłany formularz nie jest próbą `sql incjection`.

### Zadanie 3. Za pomocą dowolnego wzorca projektowego zaimplementuj mechanizm sprawdzający poprawność wyrażenia postaci:
* a + b = c (poprawne),
* (a + b = c (niepoprawne),
* a + = c (niepoprawne).

In [1]:
"""
Zadanie 3.1
Zaprojektuj klasę dla kd-drzewa.
"""
from numpy import array


class kd_Tree:
    def __init__(self, pnt, d=0):
        n = len(pnt)
        m = n // 2
        self.d = d
        pnt.sort(key=lambda x: x[d])
        d = (d + 1) % len(pnt[0])
        self.point = pnt[m]
        self.left = self.right = None
        if m > 0:
            self.left = kd_Tree(pnt[:m], d)
        if n - (m + 1) > 0:
            self.right = kd_Tree(pnt[m + 1:], d)

    def searchRange(self, arr):
        pnt = self.point
        sp = pnt[self.d]
        min, max = arr[self.d]
        if located(pnt, arr):
            yield pnt
        if self.left is not None and sp >= min:
            yield from self.left.searchRange(arr)
        if self.right is not None and sp <= max:
            yield from self.right.searchRange(arr)


def located(pnt, arr):
    return all(arr[:, 0] <= pnt) and all(pnt <= arr[:, 1])


points = [(0, 4), (7, 5), (1, 6), (11, 10), (8, 9)]
tree = kd_Tree(points)

print(list(tree.searchRange(array([[1, 6], [5, 15]]))))
print(list(tree.searchRange(array([[0, 4], [1, 6]]))))

[(1, 6)]
[(1, 6), (0, 4)]


In [2]:
""" Zadanie 2.
 Zaimplementuj wzorzec projektowy łańcuch odpowiedzialności na przykładzie obsługi żądania http (symulacja),
 w którym przed faktycznym kodem obsługi błędu ma zostać sprawdzone czy użytkownik może wysłać danego typu żądanie,
 czy żądanie nie dotyczy pliku, czy liczba żądań na minutę nie jest przekroczona,
 czy liczba żądań na minutę nie jest przekroczona dla zalogowanego użytkownika,
 czy przesłany formularz nie jest próbą sql incjection."""

class HandlerInterface:

    def next(self, nexto):
        self.nexto = nexto
        return nexto

    def handle(self, request, sender):
        if self.nexto:
            return self.nexto.handle(request, sender)


class UserRequestHandler(HandlerInterface):
    def __init__(self):
        return super().__init__()

    def handle(self, request, sender):
        if (sender.acc < 2):
            print("Zbyt niskie uprawnienia!")
        else:
            return super().handle(request, sender)


class IsFileHandler(HandlerInterface):
    def __init__(self):
        return super().__init__()

    def handle(self, request, sender):
        if ("." in request):
            print("Zapytanie dotyczy pliku")
        else:
            return super().handle(request, sender)



class RequestTimesHandler(HandlerInterface):
    def __init__(self):
        return super().__init__()

    def handle(self, request, sender):
        if (requestTimes < 100):
            return super().handle(request, sender)
        else:
            print("Przekroczono liczbę zapytań")


class UserRequestTimesHandler(HandlerInterface):
    def __init__(self):
        return super().__init__()

    def handle(self, request, sender):
        if (sender.req < 100):
            return super().handle(request, sender)
        else:
            print("Przekroczona liczba zapytań od zalogowanego użytkownika")


class SQLInjectionHandler(HandlerInterface):
    def __init__(self):
        return super().__init__()

    def handle(self, request, sender):
        if ("SQL" in request):
            print("Próba zapytania SQL")
        else:
            return super().handle(request, sender)



class AllRightHandler(HandlerInterface):
    def __init__(self):
        return super().__init__()

    def handle(self, request, sender):
        print("Witamy")


class Klient:
    handler: HandlerInterface = UserRequestHandler()
    req = 0
    acc = 0

    def __init__(self, access, requests):
        self.acc = access
        self.req = requests

    def Request(self, request):
        self.handler.next(IsFileHandler()).next(RequestTimesHandler()).next(UserRequestTimesHandler()).next(
            SQLInjectionHandler()).next(AllRightHandler())
        self.handler.handle(request, self)


k = Klient(2, 1)
k1 = Klient(2, 1)
k3 = Klient(1, 1)

requestTimes = 30
k.Request("Franek Dolas")
k.Request("SQL")
k.Request("https://bykowski.pl/wzorce-projektowe/")
requestTimes = 150
k1.Request("Patryk S")
k1.Request("https://bykowski.pl/wzorce-projektowe/")
requestTimes = 15
k1.Request("https://www.gry-online.pl/")
k3.Request("https://www.gry-online.pl/")

Witamy
Próba zapytania SQL
Zapytanie dotyczy pliku
Przekroczono liczbę zapytań
Zapytanie dotyczy pliku
Zapytanie dotyczy pliku
Zbyt niskie uprawnienia!


In [3]:
'''Zadanie 3
 Zadanie 3. Za pomocą dowolnego wzorca projektowego zaimplementuj mechanizm sprawdzający poprawność wyrażenia postaci:
 a + b = c (poprawne),
 (a + b = c (niepoprawne),
 a + = c (niepoprawne)'''



class Stacklist:
    def __init__(self):
        self.list = []

    def push(self, item):
        self.list.append(item)

    def pop(self):
        return self.list.pop()

    def isEmpty(self):
        return len(self.list) == 0


def checkOperators(exp):
    unCorrect = "Niepoprawne"
    left_right = exp.split('=')
    if len(left_right) != 2:
        return unCorrect
    for z in left_right:
        fOperand = Stacklist()
        operator = Stacklist()
        for c in z:
            if c.isalpha():
                if fOperand.isEmpty():
                    fOperand.push(c)
                else:
                    if not operator.isEmpty():
                        op = operator.pop()
                        a = fOperand.pop()
                        fOperand.push(op + a + c)
                    else:
                        return unCorrect
            elif c in "+-/*":
                if fOperand.isEmpty():
                    return unCorrect
                else:
                    operator.push(c)
        if operator.isEmpty() is not True:
            return unCorrect
    unCorrect = "Poprawne"
    return unCorrect


def checkParentheses(exp):
    unCorrect = "Niepoprawne"
    parentheses = Stacklist()
    for c in exp:
        if c == '(':
            parentheses.push(c)
        elif c == ')':
            if parentheses.isEmpty():
                return unCorrect
            else:
                parentheses.pop()
        elif c == '=':
            if not parentheses.isEmpty():
                return unCorrect
    return checkOperators(exp)


print(checkParentheses('a + b = c'))
print(checkParentheses('(a + b = c'))
print(checkParentheses('a + = c'))


Poprawne
Niepoprawne
Niepoprawne
